In [ ]:
import tensorflow as tf
import keras
import numpy as np
import GA
import pandas as pd
import random as rn
import time

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
#Reducing the number of rows

a=x_train[y_train==0][:500]
a1=x_test[y_test==0][:100]

i=1
while i<10:
    a=np.append(a,x_train[y_train==i][:500],axis=0)
    a1=np.append(a1,x_test[y_test==i][:100],axis=0)
    i=i+1
    
b=np.array([np.full((500,),0),np.full((500,),1),np.full((500,),2),np.full((500,),3),np.full((500,),4),np.full((500,),5),np.full((500,),6),np.full((500,),7),np.full((500,),8),np.full((500,),9)])
b=b.reshape((5000,))

b1=np.array([np.full((100,),0),np.full((100,),1),np.full((100,),2),np.full((100,),3),np.full((100,),4),np.full((100,),5),np.full((100,),6),np.full((100,),7),np.full((100,),8),np.full((100,),9)])
b1=b1.reshape((1000,))

x_,y_=a,b
x1_,y1_=a1,b1

s = np.arange(x_.shape[0])
s1 = np.arange(x1_.shape[0])
np.random.shuffle(s)
np.random.shuffle(s1)
x_train=x_[s]
y_train=y_[s]
x_test=x1_[s1]
y_test=y1_[s1]

In [ ]:
x_train=x_train.reshape((x_train.shape[0],x_train.shape[1]*x_train.shape[2]))
x_test=x_test.reshape((x_test.shape[0],x_test.shape[1]*x_test.shape[2]))

In [ ]:


from sklearn.preprocessing import StandardScaler
X_train = StandardScaler().fit_transform(x_train)
X_test = StandardScaler().fit_transform(x_test)
#print(standardized_data.shape)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=200).fit(pd.DataFrame(X_train))
pca_2d = pca.transform(pd.DataFrame(X_train))
X_train = np.array(pca_2d)

In [ ]:
X_train.shape

In [ ]:
#ANN

def sigmoid(inpt):
    return 1.0 / (1.0 + np.exp(-1 * inpt))

def relu(inpt):
    result = inpt
    result[inpt < 0] = 0
    return result

def predict_outputs(weights_mat, data_inputs, data_outputs, activation="relu"):
    predictions = np.zeros(shape=(data_inputs.shape[0]))
    for sample_idx in range(data_inputs.shape[0]):
        r1 = data_inputs[sample_idx, :]
        for curr_weights in weights_mat:
            r1 = np.matmul(r1,curr_weights)
            if activation == "relu":
                r1 = relu(r1)
            elif activation == "sigmoid":
                r1 = sigmoid(r1)
        predicted_label = np.where(r1 == np.max(r1))[0][0]
        predictions[sample_idx] = predicted_label
    correct_predictions = np.where(predictions == data_outputs)[0].size
    accuracy = (correct_predictions / data_outputs.size) * 100
    return accuracy, predictions

def fitness(weights_mat, data_inputs, data_outputs, activation="relu"):
    accuracy = np.empty(shape=(weights_mat.shape[0]))
    for sol_idx in range(weights_mat.shape[0]):
        curr_sol_mat = weights_mat[sol_idx, :]
        accuracy[sol_idx] , abc = predict_outputs(curr_sol_mat, data_inputs, data_outputs, activation="relu")
    return accuracy

In [ ]:
#GA
import random

# Converting each solution from matrix to vector.
def mat_to_vector(mat_pop_weights):
    pop_weights_vector = []
    for sol_idx in range(mat_pop_weights.shape[0]):
        curr_vector = []
        for layer_idx in range(mat_pop_weights.shape[1]):
            vector_weights = np.reshape(mat_pop_weights[sol_idx, layer_idx], newshape=(mat_pop_weights[sol_idx, layer_idx].size))
            curr_vector.extend(vector_weights)
        pop_weights_vector.append(curr_vector)
    return np.array(pop_weights_vector)

# Converting each solution from vector to matrix.
def vector_to_mat(vector_pop_weights, mat_pop_weights):
    mat_weights = []
    for sol_idx in range(mat_pop_weights.shape[0]):
        start = 0
        end = 0
        for layer_idx in range(mat_pop_weights.shape[1]):
            end = end + mat_pop_weights[sol_idx, layer_idx].size
            curr_vector = vector_pop_weights[sol_idx, start:end]
            mat_layer_weights = np.reshape(curr_vector, newshape=(mat_pop_weights[sol_idx, layer_idx].shape))
            mat_weights.append(mat_layer_weights)
            start = end
    return np.reshape(mat_weights, newshape=mat_pop_weights.shape)

In [10]:
## Creating the initial population.
pop_size=20
n_gen=1000
initial_pop_weights = []
for curr_sol in range(pop_size):
    
    HL1_neurons = 150
    input_HL1_weights = np.random.uniform(low=-0.1, high=0.1, 

                                             size=(X_train.shape[1], HL1_neurons))

    HL2_neurons = 60
    HL1_HL2_weights = np.random.uniform(low=-0.1, high=0.1, 

                                             size=(HL1_neurons, HL2_neurons))

    output_neurons = 10
    HL2_output_weights = np.random.uniform(low=-0.1, high=0.1, 

                                              size=(HL2_neurons, output_neurons))

    initial_pop_weights.append(np.array([input_HL1_weights, 

                                                HL1_HL2_weights, 

                                                HL2_output_weights]))
pop_weights_mat = np.array(initial_pop_weights)
pop_weights_vector = mat_to_vector(pop_weights_mat)

f=fitness(pop_weights_mat,X_train,y_train)


In [11]:
f1=np.array(f)

In [ ]:
for i in range(n_gen):
    
    
    parents=GA.tournament_selection(pop_weights_vector,f) 
    f1=f
    
    child=np.full((parents.shape),np.nan)
    
    #CROSSOVERS
    child=GA.sbx_real(parents,1,10)

    #MUTATIONS
    child=GA.mut_poly(child,50,-0.1,0.1,0.05)
    child=child.astype('float')
    #fitness
    f=fitness(np.array(vector_to_mat(child,pop_weights_mat)),X_train,y_train)
    
    con=np.concatenate((child,f.reshape((f.shape[0]),1)),axis=1)
    con=con[con[:,-1].argsort()][::-1]
    child=con[:,:-1]
    
    child[10:,:]=parents[:10,:]
    f[10:]=f1[:10]
    
    pop_weights_vector=child
    
    print("gen:",i+1, "child",child)
    print("fitness:",f,"Best Fitness",max(f))
    